In [1]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [3]:
d_tr = pd.read_csv('/Users/antirrabia/Documents/01-GitHub/DataMining-_-/CSV/Titanic/train.csv', index_col=0)
d_te = pd.read_csv('/Users/antirrabia/Documents/01-GitHub/DataMining-_-/CSV/Titanic/test.csv', index_col=0)

In [4]:
d_tr.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
X = d_tr.drop(columns=['Survived', 'Ticket', 'Cabin']).copy()
y = d_tr.Survived.copy()

In [22]:
age_np = X.Age.to_numpy()
type(age_np)

numpy.ndarray

In [25]:
#     s.loc[(s.Age <= 13), 'Age'] = 1
#     s.loc[(s.Age > 13) & (s.Age <= 18), 'Age'] = 2
#     s.loc[(s.Age > 18) & (s.Age <= 30), 'Age'] = 3
#     s.loc[(s.Age > 30) & (s.Age <= 50), 'Age'] = 4
#     s.loc[(s.Age > 50), 'Age'] = 5

len(age_np[(age_np > 13) & (age_np <= 18)])

68

In [18]:
impute_age = ColumnTransformer([ ('age', SimpleImputer(), ['Age']), 
#                              ('embarked', SimpleImputer(), ['Embarked'])
                           ])

age_trans = Pipeline([('impute', impute_age), 
                      ('encode', age_func)
                     ])

# impute = ColumnTransformer([('num_impute', SimpleImputer(strategy='mean'), ['Age']), 
#                             ('cab_impute', SimpleImputer(strategy='constant'), ['Cabin']), 
#                             ('emb_impute', SimpleImputer(strategy='most_frequent'), ['Embarked'])], 
#                              remainder='passthrough' )

In [19]:
age_trans.fit_transform(X.copy()).head(25)

<class 'numpy.ndarray'>


AssertionError: 

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)

param_grid = [{ #regular grid search took about 1 ahour with KNNImputer
            'model__n_estimators': [10, 100, 1000],
            'model__learning_rate': [0.001, 0.01, 0.1],
            'model__subsample': [0.5, 0.7, 1.0],
            'model__max_depth': [3, 7, 9],
    
#             'imputer__imputer__strategy': ['mean', 'median', 'constant', 'most_frequent']
            'imputer__num_imputer__strategy': ['mean'], # , median, constant, most_frequent
            'imputer__cab_imputer__strategy': ['constant', 'most_frequent'], # constant, most_frequent
            'imputer__cab_imputer__strategy': ['most_frequent']
              }]

In [ ]:
#pipeline = Pipeline([('clean_d', cleanData)], verbose=True)

# classifier = Pipeline([('imputer', impute), 
#                      ('embarked', codeEmbarked_toPipeline_trans),
#                      ("titles", getTitles_trans),
#                      ("fare", fare_trans),
#                      ('age', age_trans),
#                      ('family', familySize_trans), 
#                      ('oneHot', sexTitle_to_OneHot_trans), 
#                      ('model', model)])

cleaner = Pipeline([ 
                     ('imputer', impute), 
    ('to_df', toDF_trans)
                     ('embarked', codeEmbarked_trans), # para clean: codeEmbarked_trans
                     ("titles", getTitles_trans),
#                     ("fare", fare_trans),
#                      ('age', age_trans),
#                      ('family', familySize_trans), 
#                      ('oneHot', sexTitle_to_OneHot_trans) 
                   ])

In [17]:
def to_df(a):
    ''' It recive a numpy array and turning into a data frame '''
    
    df = pd.DataFrame(a.copy(), 
                      columns=['Age', 'Cabin', 'Embarked', 'Pclass', 
                               'Name', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Fare']
                     )
    
    return df


def fare(df):
    #s['FareBand'] = pd.qcut(s['Fare'], 4)
    
    s = df.copy()
    s.loc[ s['Fare'] <= 7.91, 'Fare'] = 0
    s.loc[(s['Fare'] > 7.91) & (s['Fare'] <= 14.454), 'Fare'] = 1
    s.loc[(s['Fare'] > 14.454) & (s['Fare'] <= 31), 'Fare']   = 2
    s.loc[ s['Fare'] > 31, 'Fare'] = 3

    #Cambiar tipo de columna
    s['Fare'] = s['Fare'].astype(int)
    
    return s

def age(df): #if it uses 2 times it will set all values to 1.0
            #recive a copy
            #does not touch nan values
    print(type(df))
    assert 0
    s = df.copy()
    s.loc[(s.Age <= 13), 'Age'] = 1
    s.loc[(s.Age > 13) & (s.Age <= 18), 'Age'] = 2
    s.loc[(s.Age > 18) & (s.Age <= 30), 'Age'] = 3
    s.loc[(s.Age > 30) & (s.Age <= 50), 'Age'] = 4
    s.loc[(s.Age > 50), 'Age'] = 5
    
    return s

def familySize(df):
    
    s = df.copy()
    s['FamilySize'] = s['SibSp'] +  s['Parch'] + 1
    
    s['FamilySize'] = s['FamilySize'].astype(int)
    #s['Survived'] = s['Survived'].astype(int)
    
    s['IsAlone'] = 0
    s.loc[s['FamilySize'] == 1, 'IsAlone'] = 1
    
    return s.drop(columns=['SibSp', 'Parch', 'FamilySize']).copy()


def getTitles(df):
    #str.extract(' ([A-Za-z]+)\.')
    #print(type(s))
    
    s = df.copy()
    #print(s.sample(7))
    #assert 0
    s['Title'] = s['Name'].str.extract(' ([A-Za-z]+)\.')
    s['Title'] = s['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr',                                       
                                    'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
    s['Title'] = s['Title'].replace(['Mlle', 'Ms'], 'Miss')
    s['Title'] = s['Title'].replace(['Mme'], 'Mrs')
    
    #print(s.sample(7))
    #assert 0
    return s.drop(columns='Name').copy()

def codeEmbarked(df): #Usado en Cleaner nada mas
    '''   '''
    
    s = df.copy()                          #le quito 'int' para que pueda manejar nan
    s['Embarked'] = s['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2})#.astype('int') 
    
    return s


def sexTitle_to_OneHot(df):
    '''This version works when you firts split the data set into X, y 
       and dropping 'Survived' from X
    '''
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    
    
    #d_clean.drop(columns=['Ticket', 'Cabin'], inplace=True)
    s = df.drop(columns=['Ticket', 'Cabin']).copy()
    #print(s.head(16))
    #print(s.columns)
    #s.columns
        
    cat_features = ['Sex', 'Title']
    
    #res_features = ['Age', 'Embarked', 'Pclass', 'Fare', 'IsAlone']
    
    # Pclass  Age  Fare  Embarked  IsAlone
    res_features = ['Pclass', 'Age', 'Fare', 'Embarked', 'IsAlone']
    
    transformer = ColumnTransformer([('cat', encoder, cat_features)], remainder='passthrough')
    
    transformer.fit(s)
    
    tranformed_names = transformer.named_transformers_['cat'].get_feature_names_out(cat_features)
    #tranformed_names
    
    new_names = np.append(tranformed_names, res_features)
    
    #print(new_names)
    #assert 0
    
    return pd.DataFrame(transformer.transform(s), columns=new_names)


def get_x_y(df, y):
    #resives an copy of a df and return X_tr, y_tr
    X_tr = df.drop(columns=[y])
    y_tr = df[y]
    
    return X_tr, y_tr


codeEmbarked_trans = FunctionTransformer(codeEmbarked, validate=False)
getTitles_trans = FunctionTransformer(getTitles, validate=False)
sexTitle_to_OneHot_trans = FunctionTransformer(sexTitle_to_OneHot, validate=False)
fare_trans = FunctionTransformer(fare, validate=False)
age_func = FunctionTransformer(age, validate=False)
familySize_trans = FunctionTransformer(familySize, validate=False)
toDF_trans = FunctionTransformer(to_df, validate=False)


